# Analyzing borrowers’ risk of defaulting

# Contents <a id='back'></a>

* [Stage 1. Introduction](#intro)
* [Stage 2. Data exploration](#data_review)
    * [2.1 Conclusion](#conc_2)
* [Stage 3. Data transformation](#data_transformation)
    * [3.1 Unique values](#unique_values)
    * [3.2 Duplicates](#duplicates)
    * [3.3 Misising values](#missing_values)
* [Stage 4. Categorization of data](#categorization_data)
* [Stage 5. Checking the hypotheses](#hypotheses)
    * [5.1 Hypothesis 1: Is there a correlation between having children and paying back on time?](#h1)
    * [5.2 Hypothesis 2: Is there a correlation between family status and paying back on time?](#h2)
    * [5.3 Hypothesis 3: Is there a correlation between income level and paying back on time?](#h3)
    * [5.4 Hypothesis 4: How does credit purpose affect the default rate?](#h4)
* [Stage 6. Findings](#end)

## Stage 1. Introduction <a id='intro'></a>
The primary purpose of this project is to investigate the potential impact of two demographic factors, namely marital status and the number of children, on the likelihood of loan default among the bank's customers. The goal is to contribute valuable insights to the bank's credit-scoring process, enhancing its ability to evaluate the creditworthiness of applicants. By understanding the relationships between these demographic variables and loan default, the bank can make more informed decisions and refine its lending practices.

### Goal: 
Test two hypotheses:
1. Customer's marital status affecting the probability of them in defaulting on a loan. 
2. Number of children affecting the probability of them in defaulting on a loan.  



### Open the data file and have a look at the general information.

In [5]:
import pandas as pd 
df = pd.read_csv('/datasets/credit_scoring_eng.csv') 


## Stage 2. Data exploration <a id='data_review'></a>

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [6]:
rows, columns = df.shape
print(f"The dataset has {rows} rows and {columns} columns.")
# Number of rows and columns dataset has

The dataset has 21525 rows and 12 columns.


In [7]:
N=21525
df.head(N)# First N rows

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


 - days_employed - in negative integer. Need to change the integer format to positive one.
 - education -there was difference in capital letter. Need to liase the same capital letter format in this column


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


* There are many missing values for days_employed and total_income which supposed to be 21525 rather than 19351

In [9]:
display(df[df['days_employed'].isnull()].head())
#The first column with missing data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


- The missing values at days_employed and total_income might have symmetric pattern. Further investigation should be made to identify the missing values in all rows

In [10]:
conditions = (df['days_employed'].isnull()) & (df['total_income'].isnull()) 
filtered_data = df[conditions]
num_rows_filtered = len(filtered_data) # Filtering data 
print(f"The number of rows in the filtered table is: {num_rows_filtered}")
# Number of rows with missing values in the filtered table.

The number of rows in the filtered table is: 2174


**Intermediate conclusion** 
- The number of rows in the filtered table (2,174) matches the number of missing values in the 'days_employed' and 'total_income' column. The conclusion from this match is that all the rows with missing values in the 'days_employed' column also have missing values in the 'total_income' column. The consistency in the number of missing values across these columns suggests that these missing values are likely related and may occur under similar circumstances or for a specific subset of the data.

In [12]:
missing_days_employed_data = df[df['days_employed'].isnull()]
# Filter rows where 'days_employed' is missing
display(missing_days_employed_data.describe(include='all'))
# Display information about clients with missing 'days_employed'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,2174.000000,0.0,2174.000000,2174,2174.000000,2174,2174.000000,2174,2174,2174.000000,0.0,2174
unique,NaN,NaN,NaN,12,NaN,5,NaN,2,5,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,having a wedding
freq,NaN,NaN,NaN,1408,NaN,1237,NaN,1484,1105,NaN,NaN,92
mean,0.552438,NaN,43.632015,NaN,0.800828,NaN,0.975161,NaN,NaN,0.078197,NaN,NaN
std,1.469356,NaN,12.531481,NaN,0.530157,NaN,1.418220,NaN,NaN,0.268543,NaN,NaN
min,-1.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN
25%,0.000000,NaN,34.000000,NaN,0.250000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN
50%,0.000000,NaN,43.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN
75%,1.000000,NaN,54.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN


In [13]:
percentage_missing = (df['days_employed'].isnull().sum() / len(df)) * 100
print(percentage_missing) 
# Checking distribution of missing values in days_employed's column  with whole data set

10.099883855981417


- The percentage of missing values in days_employed's column compared to the whole data set was at 10.09%

**Possible reasons for missing values in data**
- the possible reason of missing values might be due to imputation during data preprocessing
- other reason could be due to retirees where they are not currently employed

In [14]:
percentage_days_employed = (df['days_employed'].count() / len(df)) * 100
print(percentage_days_employed)
# Checking distribution of values in days_employed's column with whole data set

89.90011614401858


**Intermediate conclusion**
- the distribution of original dataset to the filtered table is the same
- the missing values due to accidental because some data such as employed had retiree where they do not have days of employed and total of income

### Conclusions <a id='conc_2'></a>
- there was a pattern of missing values which in symmetric pattern. Because there was missing/negative values in days_employed and total_income
- the missing values can be solved by making the value in days_employed which in negative values as null  

## Stage 3. Data transformation <a id='data_transformation'></a>


### Unique values <a id='unique_values'></a>

In [19]:
unique_education_values = df['education'].unique()
print("Unique values in the 'education' column:")
for value in unique_education_values:
    print(value)
# Check the unique values in the 'education' column

Unique values in the 'education' column:
Bachelor's Degree
Secondary Education
Some College
Primary Education
Graduate Degree


In [20]:
df['education'] = df['education'].replace({
    "bachelor's degree": "Bachelor's Degree",
    "secondary education": "Secondary Education",
    "BACHELOR'S DEGREE": "Bachelor's Degree",
    "SECONDARY EDUCATION": "Secondary Education",
    "SOME COLLEGE": "Some College",
    "some college": "Some College",
    "primary education": "Primary Education",
    "PRIMARY EDUCATION": "Primary Education",
    "GRADUATE DEGREE": "Graduate Degree",
    "graduate degree": "Graduate Degree"
})
#Fix manual for capital letter in 'education' column

In [21]:
unique_education_values_fixed = df['education'].unique()
print("Unique values in the 'education' column after manual fixing:")
for value in unique_education_values_fixed:
    print(value)
# Checking if there any values that need to be fixed in the 'education' column

Unique values in the 'education' column after manual fixing:
Bachelor's Degree
Secondary Education
Some College
Primary Education
Graduate Degree


In [22]:
unique_children_values = df['children'].unique()
print("Unique values in the 'children' column:")
for value in unique_children_values:
    print(value)
#Check the unique values in the 'children' column

Unique values in the 'children' column:
1
0
3
2
-1
4
20
5


- Problems: there was anomality values where the value at -1 and too high number of children at 20, this might occur due to error during data imputation
- Solution: -1 and 20 should be replaced with NaN

In [23]:
import numpy as np
df['children'] = df['children'].apply(lambda x: x if x >= 0 else np.nan) #replace negative values with NaN
df['children'] = df['children'].apply(lambda x: x if x <= 10 else np.nan) #replace high values with NaN

In [24]:
df['children'] = df['children'].astype('Int64')
#change data type to int64

unique_children_values = df['children'].unique()
print("Unique values in the 'children' column:")
for value in unique_children_values:
    print(value)
# Check the unique values in the 'children' column

Unique values in the 'children' column:
1
0
3
2
<NA>
4
5


In [25]:
unique_days_employed_values = df['days_employed'].unique()
print("Unique values in the 'days_employed' column:")
for value in unique_days_employed_values:
    print(value)
# Find unique values in days_employed

Unique values in the 'days_employed' column:
-8437.673027760233
-4024.803753850451
-5623.422610230956
-4124.747206540018
340266.07204682194
-926.1858308789184
-2879.202052139952
-152.7795691752705
-6929.865298973741
-2188.7564450779378
-4171.483646903305
-792.7018870609315
nan
-1846.6419410560736
-1844.9561821875543
-972.3644189900732
-1719.9342261530542
-2369.999720209008
400281.1369125588
-10038.818548915877
-1311.6041664289485
-253.68516578402495
-1766.6441376815046
-272.9813850334044
338551.95291136915
363548.4893483957
-529.1916346594236
-717.2743239418727
335581.6685149753
-1682.08343808475
-4649.910831728614
-1548.63754364046
-4488.067031277062
394021.0721837952
-176.21668833712155
-6448.810859676204
-597.8818273051091
-650.5877964026586
-1030.3626056014652
-1257.4961901986044
-4375.681384360526
-1362.0417275002387
-1039.4517405766287
-2262.712303593806
-2689.1372742032745
-3341.0678863600638
-1181.443228272809
353731.4323382981
-6953.631654563154
-3554.6715257531628
-4219.88338

- Problems: Based on the observation, there was anomality where value at negative values and too high as it was impossible for the days employed to be hundred of years, This might be happened due to technical or imputing error
- Solution: it could be saved by turning negative values to positive values
  Solution: change high values with NaN
  Solution: if there was missing values, Imputation with median and mean of values

In [26]:
df['days_employed'] = df['days_employed'].abs() #convert negative to positive values
df['days_employed'] = np.where(df['days_employed'] > 365 * 100, np.nan, df['days_employed']) #convert high values to NaN values

In [27]:
unique_days_employed_values = df['days_employed'].unique()
print("Unique values in the 'days_employed' column:")
for value in unique_days_employed_values:
    print(value)
# Check unique values in days_employed

Unique values in the 'days_employed' column:
8437.673027760233
4024.803753850451
5623.422610230956
4124.747206540018
nan
926.1858308789184
2879.202052139952
152.7795691752705
6929.865298973741
2188.7564450779378
4171.483646903305
792.7018870609315
1846.6419410560736
1844.9561821875543
972.3644189900732
1719.9342261530542
2369.999720209008
10038.818548915877
1311.6041664289485
253.68516578402495
1766.6441376815046
272.9813850334044
529.1916346594236
717.2743239418727
1682.08343808475
4649.910831728614
1548.63754364046
4488.067031277062
176.21668833712155
6448.810859676204
597.8818273051091
650.5877964026586
1030.3626056014652
1257.4961901986044
4375.681384360526
1362.0417275002387
1039.4517405766287
2262.712303593806
2689.1372742032745
3341.0678863600638
1181.443228272809
6953.631654563154
3554.6715257531628
4219.88338560805
3480.072792773725
2424.809748937992
335.31863038428764
4341.7867754100425
2534.462390327333
2986.20212028634
7845.6492334208915
1182.5477437277514
118.8914114245009

In [28]:
unique_dob_years_values = df['dob_years'].unique()
print("Unique values in the 'dob_years' column:")
for value in unique_dob_years_values:
    print(value)
# Check the `dob_years` if there any unique values

Unique values in the 'dob_years' column:
42
36
33
32
53
27
43
50
35
41
40
65
54
56
26
48
24
21
57
67
28
63
62
47
34
68
25
31
30
20
49
37
45
61
64
44
52
46
23
38
39
51
0
59
29
60
55
58
71
22
73
66
69
19
72
70
74
75


In [29]:
percentage_zero_dob_years = (df['dob_years'] == 0).mean() * 100
print(f"Percentage of 0 values in the 'dob_years' column: {percentage_zero_dob_years:.2f}%")
#Percentage of 0 values in the dob_years

Percentage of 0 values in the 'dob_years' column: 0.47%


- Problem: There was a value of 0 in the dob_years column
- Solution: change the value of 0 to NaN values

In [30]:
df['dob_years'] = df['dob_years'].replace(0, np.nan) #
# Replace 0 valueS with NaN


In [31]:
df['dob_years'] = df['dob_years'].astype('Int64')
#change data type to int64

unique_dob_years_values = df['dob_years'].unique()
print("Unique values in the 'dob_years' column:")
for value in unique_dob_years_values:
    print(value)
# Check the result if there any 0 values again

Unique values in the 'dob_years' column:
42
36
33
32
53
27
43
50
35
41
40
65
54
56
26
48
24
21
57
67
28
63
62
47
34
68
25
31
30
20
49
37
45
61
64
44
52
46
23
38
39
51
<NA>
59
29
60
55
58
71
22
73
66
69
19
72
70
74
75


In [32]:
unique_family_status_values = df['family_status'].unique()
print("Unique values in the 'family_status' column:")
for value in unique_family_status_values:
    print(value)
# Check the `family_status` if there any unique values

Unique values in the 'family_status' column:
married
civil partnership
widow / widower
divorced
unmarried


- Problem: there was no issues with the data set contained in family_status

In [33]:
unique_gender_values = df['gender'].unique()
print("Unique values in the 'gender' column:")
for value in unique_gender_values:
    print(value)
# Check the `gender` if there any unique values

Unique values in the 'gender' column:
F
M
XNA


- Problem: there was XNA value in gender column
- Solution: Replace XNA value with NaN

In [34]:
df['gender'] = df['gender'].replace('XNA', np.nan)
# Replace 'XNA' with 'NaN'

In [35]:
unique_gender_values = df['gender'].unique()
print("Unique values in the 'gender' column:")
for value in unique_gender_values:
    print(value)
# Check the result if there other values again

Unique values in the 'gender' column:
F
M
nan


In [36]:
unique_income_type_values = df['income_type'].unique()
print("Unique values in the 'income_type' column:")
for value in unique_income_type_values:
    print(value)
# Check the `income_type` if there any unique values

Unique values in the 'income_type' column:
employee
retiree
business
civil servant
unemployed
entrepreneur
student
paternity / maternity leave


- Problem: there was no issues with the data set contained in income_type

### Removing duplicates <a id='duplicates'></a>

In [39]:
duplicates = df.duplicated() 
# check the duplicates for entire Data Frame

duplicate_rows = df[duplicates]
print("Duplicate rows in the DataFrame:")
print(duplicate_rows)
# display rows that are duplicates

num_duplicates = duplicates.sum()
print(f"\nNumber of duplicate rows: {num_duplicates}")
# count the number of rows which contained duplicated rows 

Duplicate rows in the DataFrame:
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []

Number of duplicate rows: 0


- Problem: There was number of 71 rows which had duplicate data set
- Solution: Remove duplicated rows

In [40]:
df = df.drop_duplicates()
# Remove duplicates and update the DataFrame

In [41]:
num_duplicates = duplicates.sum()
print(f"\nNumber of duplicate rows: {num_duplicates}")
# check again by count the number of rows which contained duplicated rows 


Number of duplicate rows: 0


In [42]:
df_no_duplicates = df.drop_duplicates() #remove duplicates
size_after_remove_duplicates = df_no_duplicates.shape #check the size of data set after remove duplicates

print("Size of the dataset after removing duplicates:", size_after_remove_duplicates)

Size of the dataset after removing duplicates: (21454, 12)


- Before duplicate: The dataset has 21525 rows and 12 columns.
- After duplicate: The dataset has 21454 rows and 12 columns.

In [43]:
percentage_change_values= (21454/21525)*100
print(percentage_change_values)

99.67015098722416


### Working with missing values  <a id='missing_values'></a>

In [44]:
education_dict = {0: 'Bachelor\'s Degree', 1: 'Secondary Education', 2: 'Some College', 3: 'Primary Education'}
family_status_dict = {0: 'Married', 1: 'Civil Partnership', 2: 'Unmarried', 3: 'Divorced', 4: 'Widow / Widower'}
income_type_dict = {0: 'Employee', 1: 'Businessman', 2: 'Retiree', 3: 'Civil Servant', 4: 'Student'}
# Find the dictionaries

**Restoring missing values in `total_income`**

- Problem: there were two columns which have values missing which are total_income and days_employed.
- Solution: Address total income missing values, create and age category for clients, Create a new column with the age category


In [45]:
def calculate_age_category(age):
    if pd.isnull(age):  # Check if age is missing (NaN)
        return 'Unknown'
    if age < 18:
        return 'Young'
    elif 18 <= age < 30:
        return 'Youth'
    elif 30 <= age < 45:
        return 'Adults'
    elif 45 <= age < 60:
        return 'Senior'
    else:
        return 'Elder'    # create for age category for clients

df['age_category'] = df['dob_years'].apply(calculate_age_category)
# Create a new 'age_category' column

In [46]:
print(df[['dob_years', 'age_category']].head(20))
# Display the DataFrame with the new 'age_category' column

    dob_years age_category
0          42       Adults
1          36       Adults
2          33       Adults
3          32       Adults
4          53       Senior
5          27        Youth
6          43       Adults
7          50       Senior
8          35       Adults
9          41       Adults
10         36       Adults
11         40       Adults
12         65        Elder
13         54       Senior
14         56       Senior
15         26        Youth
16         35       Adults
17         33       Adults
18         53       Senior
19         48       Senior


In [47]:
df_no_missing = df.dropna()
# Create a new DataFrame without missing values

print(df_no_missing.head(11))
# Display a few rows of the new DataFrame

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    Bachelor's Degree             0   
1          1    4024.803754         36  Secondary Education             1   
2          0    5623.422610         33  Secondary Education             1   
3          3    4124.747207         32  Secondary Education             1   
5          0     926.185831         27    Bachelor's Degree             0   
6          0    2879.202052         43    Bachelor's Degree             0   
7          0     152.779569         50  Secondary Education             1   
8          2    6929.865299         35    Bachelor's Degree             0   
9          0    2188.756445         41  Secondary Education             1   
10         2    4171.483647         36    Bachelor's Degree             0   
11         0     792.701887         40  Secondary Education             1   

        family_status  family_status_id gender income_type  debt  \
0      

In [48]:
mean_income_by_education = df.groupby('education')['total_income'].mean()
mean_income_by_family_status = df.groupby('family_status')['total_income'].mean()
mean_income_by_income_type = df.groupby('income_type')['total_income'].mean()
# group by 'education', 'family_status', and 'income_type' and calculate mean income

print("Mean Income by Education:")
print(mean_income_by_education)

print("\nMean Income by Family Status:")
print(mean_income_by_family_status)

print("\nMean Income by Income Type:")
print(mean_income_by_income_type)
# Display the mean income values

Mean Income by Education:
education
Bachelor's Degree      33142.802434
Graduate Degree        27960.024667
Primary Education      21144.882211
Secondary Education    24594.503037
Some College           29045.443644
Name: total_income, dtype: float64

Mean Income by Family Status:
family_status
civil partnership    26694.428597
divorced             27189.354550
married              27041.784689
unmarried            26934.069805
widow / widower      22984.208556
Name: total_income, dtype: float64

Mean Income by Income Type:
income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64


In [51]:
median_income_by_education = df.groupby('education')['total_income'].median()
median_income_by_family_status = df.groupby('family_status')['total_income'].median()
median_income_by_income_type = df.groupby('income_type')['total_income'].median()
# group by 'education', 'family_status', and 'income_type' and calculate median income

print("Median Income by Education:")
print(median_income_by_education)

print("\nMedian Income by Family Status:")
print(median_income_by_family_status)

print("\nMedian Income by Income Type:")
print(median_income_by_income_type)
# display the median income values

df.pivot_table(index=['age_category','income_type' ], columns='education', values='total_income', \
                             aggfunc='median')

Median Income by Education:
education
Bachelor's Degree      28054.5310
Graduate Degree        25161.5835
Primary Education      18741.9760
Secondary Education    21836.5830
Some College           25618.4640
Name: total_income, dtype: float64

Median Income by Family Status:
family_status
civil partnership    23186.534
divorced             23515.096
married              23389.540
unmarried            23149.028
widow / widower      20514.190
Name: total_income, dtype: float64

Median Income by Income Type:
income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64


education                                 Bachelor's Degree  Graduate Degree  \
age_category income_type                                                       
Adults       business                            33169.0400              NaN   
             civil servant                       27952.7200       17822.7570   
             employee                            26948.8590       18551.8460   
             paternity / maternity leave                NaN              NaN   
             retiree                             22993.1380              NaN   
             unemployed                                 NaN              NaN   
Elder        business                            30985.7500              NaN   
             civil servant                       29551.2285              NaN   
             employee                            27969.0405              NaN   
             retiree                             23522.6545       28334.2150   
Senior       business                            35023.4600              NaN   
             civil servant                       29026.8060              NaN   
             employee                            27427.6640       37358.5575   
             retiree                             22944.0730              NaN   
             unemployed                          32435.6020              NaN   
Unknown      business                            35921.6940              NaN   
             civil servant                       23595.8215              NaN   
             employee                            25178.0750              NaN   
             retiree                             22027.8590              NaN   
Youth        business                            28250.2840              NaN   
             civil servant                       24617.5440              NaN   
             employee                            24030.5850              NaN   
             entrepreneur                        79866.1030              NaN   
             retiree                             34394.1280              NaN   
             student                             15712.2600              NaN   

education                                 Primary Education  \
age_category income_type                                      
Adults       business                            21441.9210   
             civil servant                       21150.6960   
             employee                            19542.3265   
             paternity / maternity leave                NaN   
             retiree                             17973.2670   
             unemployed                                 NaN   
Elder        business                                   NaN   
             civil servant                              NaN   
             employee                            26188.2420   
             retiree                             16348.0735   
Senior       business                            27036.7905   
             civil servant                       16922.6250   
             employee                            20896.0125   
             retiree                             17026.0580   
             unemployed                                 NaN   
Unknown      business                                   NaN   
             civil servant                              NaN   
             employee                                   NaN   
             retiree                                    NaN   
Youth        business                            24210.1960   
             civil servant                       30563.3830   
             employee                            24583.5965   
             entrepreneur                               NaN   
             retiree                                    NaN   
             student                                    NaN   

education                                 Secondary Education  Some College  
age_category income_type                                   

- When dealing with potentially skewed or non-normally distributed data, the median may be a safer choice. Additionally,  median as a robust measure of central tendency.


In [52]:
import pandas as pd

def fill_missing_numeric(df, column, strategy='median'):
    filled_df = df.copy()
    if strategy == 'mean':
        fill_value = df[column].mean()
    elif strategy == 'median':
        fill_value = df[column].median()
    else:
        raise ValueError("Invalid strategy. Use 'mean' or 'median'.")
    
    filled_df[column] = filled_df[column].apply(lambda x: fill_value if pd.isnull(x) else x)
    #fill missing values and apply to every rows
    
    return filled_df

df_filled = fill_missing_numeric(df, 'total_income', strategy='median')
#new function for filling missing values

In [53]:
print(df_filled)
# check the function

       children  days_employed  dob_years            education  education_id  \
0             1    8437.673028         42    Bachelor's Degree             0   
1             1    4024.803754         36  Secondary Education             1   
2             0    5623.422610         33  Secondary Education             1   
3             3    4124.747207         32  Secondary Education             1   
4             0            NaN         53  Secondary Education             1   
...         ...            ...        ...                  ...           ...   
21520         1    4529.316663         43  Secondary Education             1   
21521         0            NaN         67  Secondary Education             1   
21522         1    2113.346888         38  Secondary Education             1   
21523         3    3112.481705         38  Secondary Education             1   
21524         2    1984.507589         40  Secondary Education             1   

           family_status  family_status

In [54]:
print(df_filled.isnull().sum())
# find the missing values for each column

print(df_filled.describe())
# check either count value same as number of rows

print(df_filled.head())
# check if the columns had reasonable info

children             123
days_employed       5548
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 1
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64
           children  days_employed     dob_years  education_id  \
count  21331.000000   15906.000000  21353.000000  21454.000000   
mean       0.474005    2353.015932     43.475905      0.817097   
std        0.752378    2304.243851     12.242316      0.548674   
min        0.000000      24.141633     19.000000      0.000000   
25%        0.000000     756.371964     33.000000      1.000000   
50%        0.000000    1630.019381     43.000000      1.000000   
75%        1.000000    3157.480084     53.000000      1.000000   
max        5.000000   18388.949901     75.000000      4.000000   

       family_status_id          debt   total_income  
count      2

In [55]:
missing_children = df_filled[df_filled['children'].isnull() | df_filled['dob_years'].isnull() | df_filled['gender'].isnull()]
# check rows where 'children' or 'dob_years' or 'gender' is missing

print(missing_children[['children', 'dob_years', 'gender']])
# print the information about missing values in these rows


       children  dob_years gender
99            0       <NA>      F
149           0       <NA>      F
270           3       <NA>      F
291        <NA>         46      F
578           0       <NA>      F
...         ...        ...    ...
21313         0       <NA>      M
21325      <NA>         37      F
21390      <NA>         53      M
21404      <NA>         52      M
21491      <NA>         27      F

[224 rows x 3 columns]


In [56]:
median_children = df_filled['children'].median()
df_filled['children'].fillna(median_children, inplace=True)
# fix the values in children with median

median_age = df_filled['dob_years'].median()
df_filled['dob_years'].fillna(median_age, inplace=True)
# fix the values in dob_years with median

mode_gender = df_filled['gender'].mode()[0]
df_filled['gender'].fillna(mode_gender, inplace=True)
# fix the values in mode_gender with mode

In [57]:
print(df_filled[['children', 'dob_years', 'gender']].isnull().sum())

children     0
dob_years    0
gender       0
dtype: int64


In [58]:
column_entries = df_filled[['children', 'dob_years', 'gender']].count()
print(column_entries)
# checking the number of entries in the columns

children     21454
dob_years    21454
gender       21454
dtype: int64


In [59]:
print(df_filled.isnull().sum())
# find the missing values for each column

children               0
days_employed       5548
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64


**Restoring values in `days_employed`**

In [60]:
mean_days_employed_by_children = df.groupby('children')['days_employed'].mean()
# group by 'children', and calculate mean days_employed

print("Mean Days Employed by Children:")
print(mean_days_employed_by_children)

# Display the mean days_employed values

Mean Days Employed by Children:
children
0    2485.139141
1    2162.610516
2    2129.640926
3    2176.013407
4    2179.915392
5    1432.348601
Name: days_employed, dtype: float64


In [61]:
median_days_employed_by_children = df.groupby('children')['days_employed'].median()
# group by 'children', and calculate median days_employed

print("Median Days Employed by Children:")
print(median_days_employed_by_children)

# Display the median days_employed values

Median Days Employed by Children:
children
0    1665.562427
1    1545.323220
2    1654.414416
3    1690.839628
4    1877.349159
5    1231.571486
Name: days_employed, dtype: float64


- When dealing with potentially skewed or non-normally distributed data, the median may be a safer choice. Additionally,  median as a robust measure of central tendency.

In [62]:
import pandas as pd

def fill_days_employed_by_children(df):
    filled_df = df.copy()
    
    # Calculate median 'days_employed' for each unique value in 'children'
    median_days_by_children = df.groupby('children')['days_employed'].median()

    # Fill missing values in 'days_employed' based on the corresponding 'children' value
    for child_value, median_days in median_days_by_children.items():
        filled_df.loc[(filled_df['days_employed'].isnull()) & (filled_df['children'] == child_value), 'days_employed'] = median_days
    
    return filled_df

# Apply the new function to fill missing values in 'days_employed'
df_filled = fill_days_employed_by_children(df_filled)

In [63]:
print(df_filled)
# check the function

       children  days_employed  dob_years            education  education_id  \
0             1    8437.673028         42    Bachelor's Degree             0   
1             1    4024.803754         36  Secondary Education             1   
2             0    5623.422610         33  Secondary Education             1   
3             3    4124.747207         32  Secondary Education             1   
4             0    1663.736107         53  Secondary Education             1   
...         ...            ...        ...                  ...           ...   
21520         1    4529.316663         43  Secondary Education             1   
21521         0    1663.736107         67  Secondary Education             1   
21522         1    2113.346888         38  Secondary Education             1   
21523         3    3112.481705         38  Secondary Education             1   
21524         2    1984.507589         40  Secondary Education             1   

           family_status  family_status

In [64]:
print(df_filled.isnull().sum())
# find the missing values for each column

print(df_filled.describe())
# check either count value same as number of rows

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64
           children  days_employed     dob_years  education_id  \
count  21454.000000   21454.000000  21454.000000  21454.000000   
mean       0.471287    2170.823242     43.473665      0.817097   
std        0.751071    2007.993631     12.213507      0.548674   
min        0.000000      24.141633     19.000000      0.000000   
25%        0.000000    1023.709707     33.000000      1.000000   
50%        0.000000    1663.736107     43.000000      1.000000   
75%        1.000000    2523.810846     53.000000      1.000000   
max        5.000000   18388.949901     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21454.000000  21454.000000   21454.00000

In [65]:
column_entries = df_filled[['children', 'dob_years', 'gender', 'total_income', 'days_employed']].count()
print(column_entries)
# checking the number of entries in the columns

children         21454
dob_years        21454
gender           21454
total_income     21454
days_employed    21454
dtype: int64


## Stage 4. Categorization of data <a id='categorization_data'></a>


- Answer these questions:
- Is there a connection between having kids and repaying a loan on time?
- Is there a connection between marital status and repaying a loan on time?
- Is there a connection between income level and repaying a loan on time?
- How do different loan purposes affect on-time loan repayment?

- Data were categorized based on needed purpose for further investigations

In [95]:
print("Unique values in 'children' column:")
print(df_filled['children'].unique())
# Print the values for your selected data for categorization

# Print unique values for the 'family_status' column
print("\nUnique values in 'family_status' column:")
print(df_filled['family_status'].unique())

# Print unique values for the 'income_category' column (created in the previous example)
print("\nUnique values in 'income_type' column:")
print(df_filled['income_type'].unique())

# Print unique values for the 'purpose' column
print("\nUnique values in 'purpose' column:")
print(df_filled['purpose'].unique())

Unique values in 'children' column:
<IntegerArray>
[1, 0, 3, 2, 4, 5]
Length: 6, dtype: Int64

Unique values in 'family_status' column:
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']

Unique values in 'income_type' column:
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']

Unique values in 'purpose' column:
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to g

- The main groups that could be considered to conduct hypotheses were children, family_status, income_type and purpose
- The categorization could be conducted such as below:

In [96]:

def categorize_data(df, columns):
    categorized_df = df.copy()

    for column in columns:
        if column == 'children':
            categorized_df['children_category'] = pd.cut(categorized_df['children'], bins=[-1, 0, 1, float('inf')], labels=['no children', '1 child', '2 or more children'])
        elif column == 'family_status':
            categorized_df['marital_status_category'] = categorized_df[column]
        elif column == 'income_type':
            categorized_df['income_type_category'] = categorized_df[column]
        elif column == 'purpose':
            categorized_df['purpose_category'] = categorized_df[column]

    return categorized_df

# Example usage:
topics_to_categorize = ['children', 'family_status', 'income_type', 'purpose']
df_categorized = categorize_data(df_filled, topics_to_categorize)

# Print the first few rows of the categorized DataFrame
print(df_categorized.head())

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    Bachelor's Degree             0   
1         1    4024.803754         36  Secondary Education             1   
2         0    5623.422610         33  Secondary Education             1   
3         3    4124.747207         32  Secondary Education             1   
4         0    1663.736107         53  Secondary Education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

                   purpose age_category   children_categ

- 4 reasonable category were chosen and categorized based on th enumerical data to indicate which subject had more low or high number of count value 

In [97]:
# Getting summary statistics for the column
df_categorized = categorize_data(df_filled, ['children', 'family_status', 'income_type', 'purpose'])
all_numerical_summary = df_categorized.describe()
df_categorized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   children                 21454 non-null  Int64   
 1   days_employed            21454 non-null  float64 
 2   dob_years                21454 non-null  Int64   
 3   education                21454 non-null  object  
 4   education_id             21454 non-null  int64   
 5   family_status            21454 non-null  object  
 6   family_status_id         21454 non-null  int64   
 7   gender                   21454 non-null  object  
 8   income_type              21454 non-null  object  
 9   debt                     21454 non-null  int64   
 10  total_income             21454 non-null  float64 
 11  purpose                  21454 non-null  object  
 12  age_category             21454 non-null  object  
 13  children_category        21454 non-null  category
 14  marita

- Several data types chosen to answer the hypotheses which were: 
- (1) children_category and debt
- (2) family_status and debt
- (3) income_type, total_income and debt
- (4) purpose and debt

## Stage 5. Checking the Hypotheses <a id='hypotheses'></a>


### Is there a correlation between having children and paying back on time? <a id='h1'></a>

In [89]:
# Group by 'children_category' and calculate the mean of 'debt' for each group
children_debt_analysis = df_categorized.groupby('children_category')['debt'].mean()
print("Average debt repayment rate based on the number of children:")
print(children_debt_analysis)

Average debt repayment rate based on the number of children:
children_category
no children           0.075419
1 child               0.092346
2 or more children    0.092516
Name: debt, dtype: float64


**Conclusion**
- In summary, the data suggests a trend where individuals with children, especially those with one or more children, may have a slightly higher average debt repayment rate compared to those without children. 

### Is there a correlation between family status and paying back on time? <a id='h2'></a>

In [90]:
# Group by 'marital_status_category' and calculate the mean of 'debt' for each group
marital_status_debt_analysis = df_categorized.groupby('marital_status_category')['debt'].mean()
print("Average debt repayment rate based on marital status:")
print(marital_status_debt_analysis)

Average debt repayment rate based on marital status:
marital_status_category
civil partnership    0.093471
divorced             0.071130
married              0.075452
unmarried            0.097509
widow / widower      0.065693
Name: debt, dtype: float64


**Conclusion**
- The data suggests that there is a correlation between marital status and debt repayment, with individuals in civil partnerships and unmarried individuals having higher average debt repayment rates compared to divorced, married, and widowed or widowed individuals.

### Is there a correlation between income level and paying back on time? <a id='h3'></a>

In [91]:
# Group by 'income_type_category' and calculate the mean of 'debt' for each group
income_type_debt_analysis = df_categorized.groupby('income_type_category')['debt'].mean()
print("Average debt repayment rate based on income level:")
print(income_type_debt_analysis)

Average debt repayment rate based on income level:
income_type_category
business                       0.074045
civil servant                  0.059025
employee                       0.095724
entrepreneur                   0.000000
paternity / maternity leave    1.000000
retiree                        0.056412
student                        0.000000
unemployed                     0.500000
Name: debt, dtype: float64


**Conclusion**

- The data indicates a correlation between income level and debt repayment, with employees having a higher average debt repayment rate, while civil servants and retirees show lower rates, and entrepreneurs and students having no recorded instances of debt.

### How does credit purpose affect the default rate? <a id='h4'></a>

In [93]:
# Multiply the 'debt' values by 100 to get percentage
purpose_debt_analysis = df_filled.groupby('purpose')['debt'].mean()
purpose_debt_analysis_percentage = purpose_debt_analysis * 100

# Display the results
print("Average debt repayment rate based on credit purpose (in percentage):")
print(purpose_debt_analysis_percentage)

Average debt repayment rate based on credit purpose (in percentage):
purpose
building a property                          8.723748
building a real estate                       7.692308
buy commercial real estate                   7.110439
buy real estate                              6.924316
buy residential real estate                  6.765677
buying a second-hand car                     7.531381
buying my own car                            9.108911
buying property for renting out              7.987711
car                                          8.502024
car purchase                                 9.110629
cars                                         9.205021
construction of own property                 6.614173
education                                    7.158837
getting an education                         8.371041
getting higher education                    10.798122
going to university                          8.669355
having a wedding                             8.333333
housi

**Conclusion**
- The data shows varying average debt repayment rates based on credit purposes, with higher rates observed for categories such as 'to get a supplementary education,' 'purchase of a car,' and 'buying my own car,' while lower rates are associated with purposes like 'purchase of my own house' and 'housing renovation.' These findings highlight the influence of credit purposes on default rates.


## Stage 6. General Conclusion <a id='end'></a>

Handling Missing Values:

- Missing values were present in the 'gender', 'total_income' and 'days_employed' columns.
- Missing values in 'total_income', 'days_employed' and 'gender' were filled using median and mode values.
- 'total_income' missing values were filled with the median value.
- 'days_employed' missing values were filled with the median value.

Duplicates: 
- Duplicates were removed for every rows of data set.

Categorization:
- The data was categorized based on 'children,' 'family_status,' 'income_type,' and 'purpose' for analysis of specific questions.

Correlation Analysis:
- A correlation analysis was performed to explore relationships between different variables and the likelihood of debt repayment.
- Correlation was observed between certain demographic factors and debt repayment rates.





Specific Conclusions for Posed Questions:

Connection Between Having Children and Repaying a Loan on Time:
- Individuals with no children had a lower average debt repayment rate compared to those with one or more children.

Connection Between Marital Status and Repaying a Loan on Time:
- Debt repayment rates varied across marital status categories, with civil partnerships and unmarried individuals having higher rates.

Connection Between Income Level and Repaying a Loan on Time:
- Employees had a higher average debt repayment rate, while civil servants and retirees showed lower rates. Entrepreneurs and students had no recorded instances of debt.

Impact of Credit Purpose on Default Rate:
- Credit purposes influenced default rates, with higher rates for purposes like education, car purchases, and supplementary education, and lower rates for housing-related purposes.

